**Analyzing Retail Fuel Price Dynamics in New
South Wales (2016–2025)**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
fuel_price_df = pd.read_csv("fuelPrice_NSW.csv")

1.1 data cleaning and visualisation

In [3]:
# handling missing values (there aren't any..?)
print(f"no. of missing values: \n{fuel_price_df.isnull().sum()}")

# parse PriceUpdatedDate into date/time; ensuring Price is numeric (cents per litre)
fuel_price_df["Date"] = fuel_price_df["PriceUpdatedDate"].apply(lambda row: row[0:10])
fuel_price_df["Time"] = fuel_price_df["PriceUpdatedDate"].apply(lambda row: row[11:])
# fuel_price_df.drop(["PriceUpdatedDate"], axis=1, inplace=True)

# ensure Price is numeric
fuel_price_df["Price"] = pd.to_numeric(fuel_price_df["Price"])
print(f"price is numeric -> price datatype: {fuel_price_df['Price'].dtypes} (cents/litre)")

# dropping exact duplicates
initial_len = len(fuel_price_df)
fuel_price_df.drop_duplicates(inplace=True)
final_len = len(fuel_price_df)
print(f"dropped {initial_len - final_len} exact duplicate rows")

# dropping implausible prices (< 50c/litre or > 300c/litre)
print(f"current min: {min(fuel_price_df['Price'])} | current max: {max(fuel_price_df['Price'])}")
# values justified in report
min_price_threshold = 60
max_price_threshold = 210

initial_len = len(fuel_price_df)
fuel_price_df = fuel_price_df[(fuel_price_df["Price"] >= min_price_threshold) & (fuel_price_df["Price"] <= max_price_threshold)]
final_len = len(fuel_price_df)
print(f"dropped {initial_len - final_len} implausible priced rows (< {min_price_threshold}c/l or > {max_price_threshold}c/l)")
print(f"current min: {min(fuel_price_df['Price'])} | current max: {max(fuel_price_df['Price'])}")

# aggregating to daily table per fuelcode by min price
daily_table = fuel_price_df.groupby(["Date", "FuelCode"])["Price"].min().reset_index()
daily_table.rename(columns={"Price": "MinPrice"}, inplace=True)
print(f"daily table per fuel by min price: {daily_table}")

no. of missing values: 
ServiceStationName    0
FuelCode              0
PriceUpdatedDate      0
Price                 0
Latitude              0
Longitude             0
dtype: int64
price is numeric -> price datatype: float64 (cents/litre)
dropped 2071 exact duplicate rows
current min: 55.8 | current max: 256.9
dropped 5586 implausible priced rows (< 60c/l or > 210c/l)
current min: 60.9 | current max: 209.9
daily table per fuel by min price:              Date FuelCode  MinPrice
0      2016-08-01      E10     101.9
1      2016-08-01      P95     114.9
2      2016-08-01      P98     119.9
3      2016-08-01      U91     103.9
4      2016-08-02      E10      99.9
...           ...      ...       ...
15440  2025-08-31      LPG     115.9
15441  2025-08-31      P95     178.9
15442  2025-08-31      P98     185.9
15443  2025-08-31      PDL     188.9
15444  2025-08-31      U91     163.9

[15445 rows x 3 columns]


In [ ]:
# visualising
E10_plot = daily_table[daily_table["FuelCode"]=="E10"]
U91_plot = daily_table[daily_table["FuelCode"]=="U91"]

E10_plot # e10 mentioned
U91_plot

plt.plot(E10_plot["Date"], E10_plot["MinPrice"])
plt.plot(U91_plot["Date"], U91_plot["MinPrice"])
plt.xlabel("Date")
plt.ylabel("Daily Fuel Price")
plt.legend(["E10", "U91"])
plt.show()

,Date,FuelCode,MinPrice
3,2016-08-01,U91,103.9
7,2016-08-02,U91,104.9
11,2016-08-03,U91,101.9
16,2016-08-04,U91,100.9
20,2016-08-05,U91,100.3
...,...,...,...
15423,2025-08-27,U91,159.9
15429,2025-08-28,U91,157.9
15433,2025-08-29,U91,166.9
15438,2025-08-30,U91,163.9
